In [ ]:
# Dynamic water residence time

import MDAnalysis as mda
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

# Load the trajectory
u = mda.Universe("topology", "trajectory")

# Selections
ligand = u.select_atoms("resname UNL")  # Change if ligand name differs
waters = u.select_atoms("resname SOL and name OW")  # For TIP3P, OH2 is oxygen

# Distance threshold (Å)
cutoff = 3.5

# Dictionary to store residence durations (in frames)
residence = defaultdict(list)
current_contacts = {}

# Loop through trajectory
print("Analyzing water residence times...")
for ts in u.trajectory:
    frame = ts.frame
    dists = mda.lib.distances.distance_array(waters.positions, ligand.positions, box=ts.dimensions)
    close_indices = np.where(np.any(dists < cutoff, axis=1))[0]
    close_waters = waters[close_indices]

    # Track presence
    current_ids = set(close_waters.indices)
    ended = set(current_contacts) - current_ids
    started = current_ids - set(current_contacts)

    # Finalize ended contacts
    for wid in ended:
        start_frame = current_contacts.pop(wid)
        duration = frame - start_frame
        residence[wid].append(duration)

    # Start new contacts
    for wid in started:
        current_contacts[wid] = frame

# Finalize remaining
for wid, start_frame in current_contacts.items():
    duration = u.trajectory.n_frames - start_frame
    residence[wid].append(duration)

# Prepare data
data = [(wid, np.mean(durations), len(durations)) for wid, durations in residence.items()]
df = pd.DataFrame(data, columns=["WaterAtomIndex", "MeanResidenceFrames", "NumVisits"])
df["MeanResidenceTime_ps"] = df["MeanResidenceFrames"] * u.trajectory.dt

# Save results
df.to_csv("water_residence_near_ligand.csv", index=False)
print("Saved water residence data to 'water_residence_near_ligand.csv'.")

# Plot histogram
plt.figure(figsize=(8, 6))
plt.hist(df["MeanResidenceTime_ps"], bins=40, color="cornflowerblue")
plt.xlabel("Mean Residence Time (ps)")
plt.ylabel("Water Count")
#plt.title("Water Residence Time Near Ligand")
plt.tight_layout()
plt.savefig("water_residence_histogram.png", dpi=300)
plt.show()
